# Méthode de Newton

In [ ]:
using Plots
plotly()

## Newton-Raphson en optimisation

Itération de Newton-Raphson:
$$
x_{k+1} = x_k-\frac{f'(x_k)}{f''(x_k)}
$$

Implémentons une version de base de la méthode de Newton.

In [ ]:
function Newton(f::Function, df::Function, d2f:: Function,
        xstart::Float64; verbose::Bool = false, store=false,
        δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = xstart
    if (store)
        iter = [ f(x) x ]
    end
    if (verbose)
        fx = f(x)
        println("$k. x = $x, f(x) = $fx")
    end
    dfx = df(x)
    while (abs(dfx) > δ && k < nmax)
        k += 1
        dfx = df(x)
        x = x-dfx/d2f(x)
        if (store)
            iter = [iter ; f(x) x]
        end
        if (verbose)
            fx = f(x)
            println("$k. x = $x, f(x) = $fx")
        end
    end
    
    if (store)
        return iter
    end
    
    return x
end

Considérons la fonction
$$
f(x) = -10x^2 + 4\sin(x) + x^4,
$$
ainsi que ses dérivées première et seconde.

In [ ]:
func(x) = -10x^2 + 4sin(x) + x^4
dfunc(x) = -20x + 4cos(x) + 4x^3
d2func(x) = -20 - 4sin(x) + 12x^2

In [ ]:
plot(func, -4.0, 4.0)

Si nous commençons assez proche de l'optimum global, nous pouvons trouver ce dernier.

In [ ]:
x0 = -3.0
iter = Newton(func, dfunc, d2func, x0, verbose = true, store = true)

In [ ]:
iter

Regardons graphiquement ce qui se passe.

In [ ]:
plot(func, -3.2, -2.0, label="Function")

In [ ]:
x = -3.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
m(-3.0)

In [ ]:
plot!(m, -3.2, -2.0, label="Model")
plot!(iter[1:2,2], iter[1:2,1], label="Newton step")
vline!([iter[1,2] iter[2,2]], label = "")

In [ ]:
plot(func, -3.2, -2.0, label="Function")
plot!(iter[:,2], iter[:,1], label="Newton steps")

Mais si le point de départ est mal choisi, nous pouvons ne pas converger, comme illustré ci-dessous.

In [ ]:
x0 = -4.0
Newton(func, dfunc, d2func, x0, verbose = true)

Ou encore...

In [ ]:
x0 = -2.2
Newton(func, dfunc, d2func, x0, verbose = true)

De plus, si la méthode converge, nous savons seulement qu'elle converge vers un point où la dérivée s'annule. Il pourrait s'agit d'un maximum local!

In [ ]:
x0 = 1.0
Newton(func, dfunc, d2func, x0, verbose = true)

Le modèle peut en effet être concave.

In [ ]:
x = 1.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
plot(m, -1.0, 1.5, label="Model")

Nous pouvons aussi converger vers un minimum local, mais pas global.

In [ ]:
x0 = 2.0
Newton(func, dfunc, d2func, x0, verbose = true)

In [ ]:
x = 2.0
m(y) = func(x)+dfunc(x)*(y-x)+0.5*d2func(x)*(y-x)^2
plot(m, 0.0, 3.5, label="Model")

## Méthode de la sécante

In [ ]:
function Secant(f::Function, df::Function, x0::Float64, x1::Float64;
                verbose::Bool = false, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = x0
    y = x1
    if (x0 == x1)
        println("x0 must different from x1")
        return
    end
    if (verbose)
        println("0. x0 = $x0, f($x0) = $(f(x0))")
        println("1. x1 = $x1, f($x1) = $(f(x1))")
    end
    dfx = df(x)
    dfy = df(y)
    while (abs(dfy) > δ && k < nmax)
        k += 1
        t = y
        y = y-(x-y)/(dfx-dfy)*dfy
        x = t
        if (verbose)
            println("$k. x = $y, f(x) = $(f(y))")
        end
        dfx = dfy
        dfy = df(y)        
    end
    
    return y
end

In [ ]:
Secant(func, dfunc, 1.0, 2.0, verbose = true)

## Recherche de racine avec la méthode de Newton

Itération:
$$
x_{k+1} = x_k-\frac{f(x)}{f'(x)}
$$

Considérons la fonction

In [ ]:
f(x) = x-2sin(x)

In [ ]:
df(x) = 1-2cos(x)

In [ ]:
plot(f, -5.0, 5.0)

Nous cherchons un zéro de $f(x)$.

In [ ]:
function NewtonRoot(f::Function, df::Function, xstart::Float64;
        verbose::Bool = false, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    x = xstart
    fx = f(x)
    if (verbose)
        println("$k. x = $x, f(x) = $fx")
    end
    while (abs(fx) > δ && k < nmax)
        k += 1
        dfx = df(x)
        x -= fx/df(x)
        fx = f(x)
        if (verbose)
            println("$k. x = $x, f(x) = $fx")
        end
    end
    
    return x
end

In [ ]:
x0 = 1.1
NewtonRoot(f, df, x0, verbose = true)

Cela fonctionne, mais nous étions proches d'un désastre! Observons que

In [ ]:
df(1.1)

La courbe est presque plate, et nous sommes entraînés vers un point très éloigné. Par chance, nous revenons sur nos pas, mais en utilisant beaucoup d'itérations.

Considérons à présent le point de départ

In [ ]:
x0 = π/3

La dérivée à ce point est

In [ ]:
df(x0)

La méthode de Newton donne

In [ ]:
NewtonRoot(f, df, x0, verbose = true)

345. x = -2.9320052522459207e6, f(x) = -2.932003802365324e6
346. x = -1.069636449829964e7, f(x) = -1.0696362541181946e7
347. x = -3.120687469004523e6, f(x) = -3.1206891805765317e6
348. x = -9.315795354884662e7, f(x) = -9.315795546723115e7
349. x = 1.2124930575547667e8, f(x) = 1.212493041941815e8
350. x = 6.735936010446468e7, f(x) = 6.735935869797994e7
351. x = 2.2701638415292305e8, f(x) = 2.270163860580663e8
352. x = 8.589223792051873e7, f(x) = 8.589223592594296e7
353. x = -1.482553023883456e7, f(x) = -1.4825529291543402e7
354. x = -9.456745416166501e6, f(x) = -9.456747384086024e6
355. x = 5.2454769649890475e6, f(x) = 5.2454780137624275e6
356. x = 3.3048351938438276e6, f(x) = 3.304836757025214e6
357. x = 1.6653221338388853e7, f(x) = 1.6653221268132353e7
358. x = 1.1099862606903695e7, f(x) = 1.1099862587682288e7
359. x = 2.2200750558682114e7, f(x) = 2.2200748691081412e7
360. x = -5.5857935441035524e7, f(x) = -5.585793589384712e7
361. x = -3.691062103323564e7, f(x) = -3.691062297030463e7

520. x = -2.0654814931212196e9, f(x) = -2.0654814930592864e9
521. x = -1.376767465837306e9, f(x) = -1.3767674676256049e9
522. x = -6.504470923879949e8, f(x) = -6.50447092721944e8
523. x = -1.319684763500885e9, f(x) = -1.3196847626976573e9
524. x = -8.536315555549965e8, f(x) = -8.536315571414601e8
525. x = -4.68738279129191e8, f(x) = -4.68738277589773e8
526. x = -2.162219013131646e9, f(x) = -2.1622190132925572e9
527. x = -4.3385485062544365e9, f(x) = -4.338548507570945e9
528. x = -2.607002100833024e9, f(x) = -2.607002100851477e9
529. x = -5.21422615843177e9, f(x) = -5.214226158600968e9
530. x = -3.4719869259458547e9, f(x) = -3.47198692726581e9
531. x = -2.0846186727266474e9, f(x) = -2.084618670749133e9
532. x = 8.894153773396921e8, f(x) = 8.894153793301473e8
533. x = -2.156719251656444e8, f(x) = -2.1567192546289426e8
534. x = -4.362433420870714e8, f(x) = -4.362433401327648e8
535. x = -1.3012298047471952e8, f(x) = -1.3012298112138668e8
536. x = -8.513773326621562e7, f(x) = -8.51377321633

690. x = 4.711731212472177e18, f(x) = 4.711731212472177e18
691. x = 9.890829657093124e18, f(x) = 9.890829657093124e18
692. x = 6.41931316970674e18, f(x) = 6.41931316970674e18
693. x = 1.9493668420792622e18, f(x) = 1.9493668420792622e18
694. x = 8.566377584389083e17, f(x) = 8.566377584389083e17
695. x = 1.713562970738722e18, f(x) = 1.713562970738722e18
696. x = 3.427146849127001e18, f(x) = 3.427146849127001e18
697. x = 1.8618915991457034e18, f(x) = 1.8618915991457034e18
698. x = -5.319227418514046e18, f(x) = -5.319227418514046e18
699. x = -4.696794109451778e19, f(x) = -4.696794109451778e19
700. x = -1.9754092904230568e20, f(x) = -1.9754092904230568e20
701. x = -1.2878570243845823e20, f(x) = -1.2878570243845823e20
702. x = 1.8075093604483921e21, f(x) = 1.8075093604483921e21
703. x = 1.0464663524604382e21, f(x) = 1.0464663524604382e21
704. x = -1.7791318399972934e19, f(x) = -1.7791318399972934e19
705. x = -5.637049434992707e19, f(x) = -5.637049434992707e19
706. x = -2.549644656225304e19, 

860. x = -2.0196372881267587e22, f(x) = -2.0196372881267587e22
861. x = -4.948385403787929e22, f(x) = -4.948385403787929e22
862. x = -2.71538434555032e22, f(x) = -2.71538434555032e22
863. x = -1.6286452921170972e22, f(x) = -1.6286452921170972e22
864. x = -4.763396312975515e22, f(x) = -4.763396312975515e22
865. x = -1.1783656279355981e23, f(x) = -1.1783656279355981e23
866. x = -7.661913536014524e22, f(x) = -7.661913536014524e22
867. x = -1.583360818107076e23, f(x) = -1.583360818107076e23
868. x = -1.0545184233445253e23, f(x) = -1.0545184233445253e23
869. x = -6.328688000202057e22, f(x) = -6.328688000202057e22
870. x = -1.6289664597625312e23, f(x) = -1.6289664597625312e23
871. x = -1.1065105289244079e24, f(x) = -1.1065105289244079e24
872. x = -6.95002627423812e23, f(x) = -6.95002627423812e23
873. x = -4.465731541547788e23, f(x) = -4.465731541547788e23
874. x = -2.9637287494664878e23, f(x) = -2.9637287494664878e23
875. x = -7.99295639578203e23, f(x) = -7.99295639578203e23
876. x = 2.06061

5.169201968247818e30

Nous sommes moins chanceux! En fait, en $\pi/3$,

In [ ]:
df(π/3)

La récurrence de Newton rencontre des problèmes comme nous avons une division par zéro. La fonction continue néanmoins comme, en raison des erreurs numériques, nous évitons la division par zéro. Toutefois, la méthode diverge, vu que $|x| \rightarrow \infty$.

Prenons à présent un point où la dérivée est plus importante.

In [ ]:
x0 = 4.0
NewtonRoot(f, df, x0, verbose = true)

La méthode converge à présent très rapidement, même si le point de départ était plus éloigné de la solution.

Notons que $x-2\sin x = 0$ est équivalent à $\frac{1}{\sin x} - \frac{2}{x} = 0$ si nous exigeons que $x \ne k\pi$, $k \in \mathcal{Z}$. La forme de la fonction est cependant différente autour du zéro de la fonction.

In [ ]:
g(x) = 1/sin(x) - 2/x
plot(g, 0.1, π-0.1)

La dérivée est

In [ ]:
dg(x) = 2/(x*x)-cos(x)/(sin(x)^2)
plot(dg, 0.1, π-0.1)

In [ ]:
x0 = 1.1
NewtonRoot(g, dg, x0, verbose = true)

Nous observons maintenant une convergence rapide, principalement en raison du fait que la fonction ne présente pas de parties plates.

In [ ]:
f(x) = exp(x/2)-x-1

In [ ]:
df(x) = 0.5*exp(x/2)-1

In [ ]:
x0 = 1.0
NewtonRoot(f, df, x0, verbose = true)

Vérifions les résultats obtenus avec la librairie `Roots`.

In [ ]:
using Roots

In [ ]:
y = fzero(f, 0.0, 5.0)

Cherchons à présent un zéro de la dérivée.

In [ ]:
y = fzero(df, 0.0, 5.0)

Que se passe-t-il si nous partons de cette solution?

In [ ]:
x0 = y
NewtonRoot(f, df, x0, verbose = true)

L'équation de Newton échoue en raison d'une division par zéro.

## Cycles

Considérons à présent la fonction

In [ ]:
h(x) = x^3 - 2*x + 2

In [ ]:
plot(h, -2.5, 1.5)

In [ ]:
dh(x) = 3x^2-2

In [ ]:
x0 = 0.0
NewtonRoot(h, dh, x0, verbose = true)

824. x = 1.0, f(x) = 1.0
825. x = 0.0, f(x) = 2.0
826. x = 1.0, f(x) = 1.0
827. x = 0.0, f(x) = 2.0
828. x = 1.0, f(x) = 1.0
829. x = 0.0, f(x) = 2.0
830. x = 1.0, f(x) = 1.0
831. x = 0.0, f(x) = 2.0
832. x = 1.0, f(x) = 1.0
833. x = 0.0, f(x) = 2.0
834. x = 1.0, f(x) = 1.0
835. x = 0.0, f(x) = 2.0
836. x = 1.0, f(x) = 1.0
837. x = 0.0, f(x) = 2.0
838. x = 1.0, f(x) = 1.0
839. x = 0.0, f(x) = 2.0
840. x = 1.0, f(x) = 1.0
841. x = 0.0, f(x) = 2.0
842. x = 1.0, f(x) = 1.0
843. x = 0.0, f(x) = 2.0
844. x = 1.0, f(x) = 1.0
845. x = 0.0, f(x) = 2.0
846. x = 1.0, f(x) = 1.0
847. x = 0.0, f(x) = 2.0
848. x = 1.0, f(x) = 1.0
849. x = 0.0, f(x) = 2.0
850. x = 1.0, f(x) = 1.0
851. x = 0.0, f(x) = 2.0
852. x = 1.0, f(x) = 1.0
853. x = 0.0, f(x) = 2.0
854. x = 1.0, f(x) = 1.0
855. x = 0.0, f(x) = 2.0
856. x = 1.0, f(x) = 1.0
857. x = 0.0, f(x) = 2.0
858. x = 1.0, f(x) = 1.0
859. x = 0.0, f(x) = 2.0
860. x = 1.0, f(x) = 1.0
861. x = 0.0, f(x) = 2.0
862. x = 1.0, f(x) = 1.0
863. x = 0.0, f(x) = 2.0


1.0

In [ ]:
ratio(x) = h(x)/dh(x)

In [ ]:
ratio(0.0)

In [ ]:
ratio(1.0)

La méthode cycle! Cependant, si nous changeons le point de départ, nous pouvons converger.

In [ ]:
x0 = 0.5
NewtonRoot(h, dh, x0, verbose = true)

In [ ]:
hp(x) = x^4/4-x^2+2x

plot(hp, -2.5, 1.5, xlabel="x", ylabel="f(x)", legend=false)

In [ ]:
x0 = 0.0
iter = Newton(hp, h, dh, x0, verbose = false, store = true)

In [ ]:
plot!(iter[:,2], iter[:,1], legend=false, linecolor=:blue3)

## Application: calcul de la racine carrée d'un nombre non négatif

La racine carrée d'un nombre réel non négatif peut être obtenue à l'aide de la méthode de Newton. Plus précisément, si nous cherchons la racine carrée de $s$, nous pouvons reformuler le problème comme la recherche d'un zéro de la fonction
$$
f(x) = x^2 - s
$$
En effet, si $f(x) = 0$, alors $s = x^2$, ou $x = \pm\sqrt{s}$.

Développons la récurrence de Newton pour ce problème.

Choisir un bon point de départ peut cependant être un problème. (Wikipedia) Avec $s$ exprimé en notation scientifique comme  $a\times 10^{2n}$ avec $1\leq a<100$ et $n$ entier, $\sqrt{s} \approx \sqrt{a} 10^n$. Le point de départ est souvent calculé comme
$$
\sqrt{s} =
\begin{cases}
2\times10^n & \mbox{ if } a < 10\\
6\times10^n & \mbox{ if } a \geq 10
\end{cases}
$$
Les facteurs deux et six sont utilisés car ils permettent d'approximer les moyennes géométriques de la plus petite et la plus grande valeurs pour un nombre donné de chiffres:
$$
\sqrt{\sqrt{1}\sqrt{10}} = 10^{\frac{1}{4}} \approx 2
$$
et
$$
\sqrt{\sqrt{10}\sqrt{100}} = 10^{\frac{3}{4}} \approx 6
$$

In [ ]:
sqs(x) = x*x-s

In [ ]:
dsq(x) = 2*x

Cherchons la racine de 6.

In [ ]:
s = 6.0

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, verbose = true)

Nous pouvons vérifier la solution comme

In [ ]:
x^2

Nous n'avons pas tout à fait 6. Comparons avec la routine de Julia.

In [ ]:
sqrt(6.0)

La solution est proche, mais différente. Réduisons la tolérance sous la valeur de $f$ à la dernière itération.

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, verbose = true, δ = 1e-8)

Nous sommes à présent très proches. Diminuons encore en prenant en compte le résultat de la 5e itération.

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, verbose = true, δ = 1e-16)

357. x = 2.449489742783178, f(x) = -8.881784197001252e-16
358. x = 2.449489742783178, f(x) = -8.881784197001252e-16
359. x = 2.449489742783178, f(x) = -8.881784197001252e-16
360. x = 2.449489742783178, f(x) = -8.881784197001252e-16
361. x = 2.449489742783178, f(x) = -8.881784197001252e-16
362. x = 2.449489742783178, f(x) = -8.881784197001252e-16
363. x = 2.449489742783178, f(x) = -8.881784197001252e-16
364. x = 2.449489742783178, f(x) = -8.881784197001252e-16
365. x = 2.449489742783178, f(x) = -8.881784197001252e-16
366. x = 2.449489742783178, f(x) = -8.881784197001252e-16
367. x = 2.449489742783178, f(x) = -8.881784197001252e-16
368. x = 2.449489742783178, f(x) = -8.881784197001252e-16
369. x = 2.449489742783178, f(x) = -8.881784197001252e-16
370. x = 2.449489742783178, f(x) = -8.881784197001252e-16
371. x = 2.449489742783178, f(x) = -8.881784197001252e-16
372. x = 2.449489742783178, f(x) = -8.881784197001252e-16
373. x = 2.449489742783178, f(x) = -8.881784197001252e-16
374. x = 2.449

534. x = 2.449489742783178, f(x) = -8.881784197001252e-16
535. x = 2.449489742783178, f(x) = -8.881784197001252e-16
536. x = 2.449489742783178, f(x) = -8.881784197001252e-16
537. x = 2.449489742783178, f(x) = -8.881784197001252e-16
538. x = 2.449489742783178, f(x) = -8.881784197001252e-16
539. x = 2.449489742783178, f(x) = -8.881784197001252e-16
540. x = 2.449489742783178, f(x) = -8.881784197001252e-16
541. x = 2.449489742783178, f(x) = -8.881784197001252e-16
542. x = 2.449489742783178, f(x) = -8.881784197001252e-16
543. x = 2.449489742783178, f(x) = -8.881784197001252e-16
544. x = 2.449489742783178, f(x) = -8.881784197001252e-16
545. x = 2.449489742783178, f(x) = -8.881784197001252e-16
546. x = 2.449489742783178, f(x) = -8.881784197001252e-16
547. x = 2.449489742783178, f(x) = -8.881784197001252e-16
548. x = 2.449489742783178, f(x) = -8.881784197001252e-16
549. x = 2.449489742783178, f(x) = -8.881784197001252e-16
550. x = 2.449489742783178, f(x) = -8.881784197001252e-16
551. x = 2.449

712. x = 2.449489742783178, f(x) = -8.881784197001252e-16
713. x = 2.449489742783178, f(x) = -8.881784197001252e-16
714. x = 2.449489742783178, f(x) = -8.881784197001252e-16
715. x = 2.449489742783178, f(x) = -8.881784197001252e-16
716. x = 2.449489742783178, f(x) = -8.881784197001252e-16
717. x = 2.449489742783178, f(x) = -8.881784197001252e-16
718. x = 2.449489742783178, f(x) = -8.881784197001252e-16
719. x = 2.449489742783178, f(x) = -8.881784197001252e-16
720. x = 2.449489742783178, f(x) = -8.881784197001252e-16
721. x = 2.449489742783178, f(x) = -8.881784197001252e-16
722. x = 2.449489742783178, f(x) = -8.881784197001252e-16
723. x = 2.449489742783178, f(x) = -8.881784197001252e-16
724. x = 2.449489742783178, f(x) = -8.881784197001252e-16
725. x = 2.449489742783178, f(x) = -8.881784197001252e-16
726. x = 2.449489742783178, f(x) = -8.881784197001252e-16
727. x = 2.449489742783178, f(x) = -8.881784197001252e-16
728. x = 2.449489742783178, f(x) = -8.881784197001252e-16
729. x = 2.449

888. x = 2.449489742783178, f(x) = -8.881784197001252e-16
889. x = 2.449489742783178, f(x) = -8.881784197001252e-16
890. x = 2.449489742783178, f(x) = -8.881784197001252e-16
891. x = 2.449489742783178, f(x) = -8.881784197001252e-16
892. x = 2.449489742783178, f(x) = -8.881784197001252e-16
893. x = 2.449489742783178, f(x) = -8.881784197001252e-16
894. x = 2.449489742783178, f(x) = -8.881784197001252e-16
895. x = 2.449489742783178, f(x) = -8.881784197001252e-16
896. x = 2.449489742783178, f(x) = -8.881784197001252e-16
897. x = 2.449489742783178, f(x) = -8.881784197001252e-16
898. x = 2.449489742783178, f(x) = -8.881784197001252e-16
899. x = 2.449489742783178, f(x) = -8.881784197001252e-16
900. x = 2.449489742783178, f(x) = -8.881784197001252e-16
901. x = 2.449489742783178, f(x) = -8.881784197001252e-16
902. x = 2.449489742783178, f(x) = -8.881784197001252e-16
903. x = 2.449489742783178, f(x) = -8.881784197001252e-16
904. x = 2.449489742783178, f(x) = -8.881784197001252e-16
905. x = 2.449

5.999999999999999

Le nombre d'itérations augmente considérablement.

Nous sommes peut-être aller trop loin, avec une tolérance sous la précision machine.

In [ ]:
eps()

Remontons un peu la tolérance.

In [ ]:
x = NewtonRoot(sqs, dsq, 2.0, verbose = true, δ = 1e-15)

Nous retrouvons la valeur donnée par Julia:

In [ ]:
x - sqrt(6)

Regardons avec d'autres valeurs.

In [ ]:
s = 25
x = NewtonRoot(sqs, dsq, 6.0, verbose = true, δ = 1e-15)
x^2

In [ ]:
s = 400
x = NewtonRoot(sqs, dsq, 6.0, verbose = true, δ = 1e-15)
x^2